In [22]:
import sys
import os
import sqlite3
import pandas as pd
import numpy as np

# Will do a grid search across problem size parameters for each of the the 6 chosen benchmarks...
### For each run, will save down the sqlite table on my hard disk
#### Will also create a copy of the deltas between GPU page faults to use as input to ML model

In [23]:
MAX_BYTES_GPU = 25443893248

# Profiled Using: 
- ```nsys profile --stats=true --cuda-memory-usage=true --cuda-um-gpu-page-faults=true --cuda-um-cpu-page-faults=true --output=<output_filename> ./<executable_name> [<executable_args>]*```

In [24]:
PROFILE_COMMAND = "nsys profile --stats=true --cuda-memory-usage=true --cuda-um-gpu-page-faults=true --cuda-um-cpu-page-faults=true --force-overwrite=true --output="

In [25]:
OUTPUT_DIR = "/mnt/storage/data/research/gpu_traces/"

In [26]:
ADDRESS_SEQ_DIR = "/home/shein/Documents/research/prefetching/data/gpu_traces/"

In [27]:
def generate_address_sequence(benchmark, filename):
    db_filename = OUTPUT_DIR + benchmark + filename + ".sqlite"
    con = sqlite3.connect(db_filename)
    gpu_page_fault_df = pd.read_sql_query("SELECT address FROM CUDA_UM_GPU_PAGE_FAULT_EVENTS ORDER BY start", con)
    gpu_page_fault_df["address"] = gpu_page_fault_df["address"].astype(np.uint64)
    output_address_seq_filename = ADDRESS_SEQ_DIR + benchmark + filename + ".address_buffer"
    f = open(output_address_seq_filename, 'wb')
    for i in range(len(gpu_page_fault_df["address"])):
        val = gpu_page_fault_df["address"][i]
        f.write(val.tobytes())
    f.close()

In [28]:
os.chdir("/home/shein/Documents/research/prefetching/data/chosen_benchmarks/executables")

# *Streaming*

## 2DConv

#### GPU Memory Usage: $8N_IN_J$ 

In [29]:
# Params
BENCHMARK = "2DCONV/"
EXECUTABLE_NAME = "./2DConvolution.exe"
NI = [100, 500, 1000, 2500, 5000, 10000, 25000, 50000, 56000]
NJ = [100, 500, 1000, 2500, 5000, 10000, 25000, 50000, 56000]

In [ ]:
for i in NI:
    for j in NJ:
        args = str(i) + " " + str(j)
        filename = str(i) + "_" + str(j)
        command = PROFILE_COMMAND + OUTPUT_DIR + BENCHMARK + filename + " " + EXECUTABLE_NAME + " " + args
        os.system(command)
        generate_address_sequence(BENCHMARK, filename)

## 3DConv

#### GPU Memory Usage: $8N_IN_JN_K$ 

In [11]:
# Params
NI = 0
NJ = 0
NK = 0

# *Non-Streaming*

## ATAX

#### GPU Memory Usage: $4N_XN_Y + 8N_Y + 4N_X$ 

In [10]:
# Params
NX = 0
NY = 0

## MVT

#### GPU Memory Usage: $4N(N+4)$ 

In [13]:
# Params
N = 0

## CORR

#### GPU Memory Usage: $8(M+1)(N+2)$ 

In [9]:
# Params
M = 0
N = 0

## COV

#### GPU Memory Usage: $8(M+1)(N+1) + 4(M+1)$ 

In [ ]:
# Params
M = 0
N = 0